In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms

In [ ]:
config = {
    "EPOCH" : 1,
    "BATCH_SIZE": 50,
    "DEVICE" : "cuda" if torch.cuda.is_available() else "cpu"
}

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

net = Net()
net.to(config["DEVICE"])

In [ ]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

train_set = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, 
                                        transform=transform)
train_loader = torch.utils.data.DataLoader(train_set, batch_size=config["BATCH_SIZE"],
                                         shuffle=True, num_workers=0)

test_set = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, 
                                        transform=transform)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=config["BATCH_SIZE"],
                                         shuffle=False, num_workers=0)

# Not needed because we have only 1 client
"""
trainset_len = ( len(trainset) // config["NUMBER_OF_CLIENTS"] ) * config["NUMBER_OF_CLIENTS"]
trainset = torch.utils.data.Subset(trainset, list(range(trainset_len)))

lengths = len(trainset) // config["NUMBER_OF_CLIENTS"] * np.ones(config["NUMBER_OF_CLIENTS"], dtype=int)
trainsets = torch.utils.data.random_split(dataset=trainset, lengths=lengths)
"""

In [ ]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001)

In [ ]:
for epch in range(config["EPOCH"]):
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        inputs.to(config["DEVICE"])
        labels.to(config["DEVICE"])
        
        optimizer.zero_grad()

        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backword()
        optimizer.step()

        running_loss += loss.item()
        if i % 100 == 0: 
            print(f"After {i} steps loss: {running_loss / 100}") # It print the loss every 100 steps
            running_loss = .0

In [ ]:
correct = 0
total = 0
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        inputs.to(config["DEVICE"])
        labels.to(config["DEVICE"])
        
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().items()

print(f"Accuracy = {100 * correct // total} %")